This version of Doc2VecCrunch is for posting on github. Slimmed down and cleaned up version of lengthy code.

Doc2VecCrunch uses Crunchbase-obtained lists of URLs for a specific topic/industry/keyword etc and uses the webscraped data from those URLs - typically only the front page. Thus input data file AI_totalv2.cor is a file with each line containing the text scraped from one URL (which is also listed on the line). Meanwhile, the input file AItotal.csv is the original file that is output by Crunchbase and contains various fields (see code) from the Crunchbase csv file generation procedure, for example "Full Description" and "Website." 

%matplotlib qt is used for interactive 3D plot.

In [1]:
# %matplotlib qt
%matplotlib widget

import scrapy
import logging
import json
import pandas as pd
from pandas import read_csv
import csv
import numpy as np
import ast
import logging
import os
import gensim
import logging
import smart_open
from sklearn.manifold import TSNE
import collections
import matplotlib.pyplot as plt
import scipy as sc
import sys
import re
np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_colwidth', None)

C:\Users\MStopa\AppData\Local\Continuum\anaconda3\envs\webscrape\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Set file names for train and test data

In [3]:
# startup_train_fileA contains the names of the files with 
# scraped data to be read in. In this example there is only one file.
startup_train_fileA=['AItotal_v2.cor']
nfiles=len(startup_train_fileA)
print('nfiles',nfiles)

nfiles 1


This new version of Doc2VecCrunchv2 has an array of filenames sent to read_corpus.
Similarly an array of filenames is sent to find_tokens.


read_corpus and find_tokens are routines that read in the .cor input file and find specific tokens therein, respectively. read_corpus typically tags each document (which is the scraped text from one website) with an integer - this being the way that gensim's doc2vec works.

In [4]:
def read_corpus(fnameA, tokens_only=False):
    i1=0
    for fname in fnameA: # most often only one filename in fnameA
        with smart_open.open(fname, encoding="iso-8859-1") as f:
            print('enumerate f',enumerate(f))
            for i, line in enumerate(f):
                tokens = gensim.utils.simple_preprocess(line)
                if tokens_only:
                    yield tokens
                else:
                    # For training data, add tags
#                    print('i',i,'\n tokens \n',tokens)
                    yy=gensim.models.doc2vec.TaggedDocument(tokens, [i+i1])
#                    print('i',i,'\n Tagged \n',yy)
                    yield yy
                    
            if not tokens_only:
                i1=i1+i+1 # note this is outside of i loop, i1 is for multiple files only
                flengths.append(i+1)

def find_tokens(fnameA, token1):
    ia=[]
    for fname in fnameA:
        with smart_open.open(fname, encoding="iso-8859-1") as f:
            for i, line in enumerate(f):
                if token1 in line:
#                    print("found:",token1,i)
                    ia.append(i)
                
        ianp=np.array(ia)
        return ianp

def find_tokens2(line,tokenA):
    Tlist0=[]
    for token1 in tokenA:
        if token1 in line:
            Tlist0.append(token1)
            
    return Tlist0


In [5]:
# the scraped data is simply searched for terms in the following list
search_list=['digital transformation','digitization','document processing', \
            'digital workplace','intelligent workflow','cloud based']

# AItotal.csv is Crunchbase (CB) output - want to correlate CB full description with scraped results e.g.
dataset1=read_csv('AItotal.csv', \
                      encoding='utf-8')
df=dataset1[['Full Description','Website']]

file1=open("Digitization.txt","w") # output file for search_list keyword matches, overwrite previous contents

flengths=[]
train_corpus=list(read_corpus(startup_train_fileA))
color_array=np.zeros((flengths[0],int(len(search_list))))

LL=len(search_list)
print('LL',LL)
dcolor=100/(LL+1)
for idz,search_term in enumerate(search_list):
    ianp1=find_tokens(startup_train_fileA,search_term)
#    print('ianp1',idz,idz*dcolor,search_term,ianp1)
    for ii in ianp1:
        color_array[ii,idz]=(idz+1)*dcolor

# this code produces an output file, file1, that conains the index in the
# .cor file (like 1 to 6000 for AItotal.cor), the url of that company and
# the full description from Crunchbase of that company. 
# TlistA was a test to append a multi-dimensional list to a list and I 
# don't think it worked. But it's not used.

# this code has a glitch. some lines get written to file1 twice with an integer that comes
# from god knows where. df4 gets somthing strange in it.
# Note: this occurs because some companies are double listed in AItotal.csv

# this section loops through the scraped text in startup_train_fileA.cor and finds
# the keywords from the current search_list. outputs to file1 for all lines 
# (i.e. companies) that have at least one keyword in their webpage.

for fname in startup_train_fileA:  # in some cases startup_train_fileA is a list of files
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            Tlist=find_tokens2(line,search_list) # find_tokens2 does one *line*, find_tokens loops through multiple files
            
            if Tlist != []:
                url=re.findall(r'(https?://\S+)', line)[0] # regex to extract url from line
# use only first found url [0]
                urlS=''.join(url)
                print('url',url,'urlS',urlS)
                df2=df[df['Website'].str.match(urlS,na=False)]
                df3=df2['Full Description']
                df4=df3.to_string(header="False",index="False")
                print('i',i,urlS,Tlist,'\n',df4[5:]) # output to terminal if url contains a keyword
                out1='i=' + str(i) + ' ' + urlS + ' ' + str(Tlist) + df4[5:]
                file1.write(out1 + '\n') # this goes to Digitization.txt currently
                
file1.close()


enumerate f <enumerate object at 0x000001E21FDCC750>
LL 6
url https://terrascale.org/ urlS https://terrascale.org/
i 20 https://terrascale.org/ ['digital transformation'] 
  TerraScale has assembled and owns a valuable portfolio of intellectual property.  This IP includes: 1. NSA level-accredited cybersecurity  2. Green Consultancy practice 3. MedTech 4. Sustainable blockchain capability Together, this IP forms the basis of TerraScale’s Green Platform, the “IQ Platform.” This platform enables our company to take a completely unique and unprecedented position in the market.  The IQ Platform creates a solution within the green cloud that allows us to create new applications and new IP, while creating an environment to securely manage sensitive medical and patient data as well as many other verticals. This program is “tech agnostic” part of our commitment to ensuring the platform remains ethical, transparent and creates value for its users.  The Green Consultancy Practice will be the poin

url https://sonline.us/pages/main/home urlS https://sonline.us/pages/main/home
i 952 https://sonline.us/pages/main/home ['digital transformation', 'cloud based'] 
 s([], )
url https://zerosystems.com urlS https://zerosystems.com
i 989 https://zerosystems.com ['digital transformation'] 
 s([], )
url https://www.tourmalinelabs.com/ urlS https://www.tourmalinelabs.com/
i 993 https://www.tourmalinelabs.com/ ['digital transformation'] 
    Tourmaline Labs, Inc. is a company specializing in the development of cutting-edge software that will collect telematics data through the use of a smart phone. Established in 2014 in San Diego, California, Tourmaline Labs has a team of world-class programmers, and aims to become a leader in providing solutions for mobile phone manufacturers, carriers and app development companies in the U.S. and Europe. The company holds two U.S. patents, which are related to telematics and drive-data collection using a mobile device.
url https://www.ripcord.com urlS http

i 1457 https://www.evolvtechnology.com/ ['digital transformation'] 
 s([], )
url https://nference.com/ urlS https://nference.com/
i 1465 https://nference.com/ ['digitization'] 
 s([], )
url https://indicodata.ai/ urlS https://indicodata.ai/
i 1472 https://indicodata.ai/ ['digital transformation', 'digitization'] 
 s([], )
url https://cappasity.com/ urlS https://cappasity.com/
i 1494 https://cappasity.com/ ['digitization'] 
 s([], )
url https://www.iterate.ai/ urlS https://www.iterate.ai/
i 1521 https://www.iterate.ai/ ['digital transformation'] 
    Iterate's platform includes two clouds: Execution and Evaluation. Organizations use Iterate to simplify and speed up the complex, ambiguous innovation process. We enable programmatic innovation. Iterate built a patented low-code platform(Interplay) with 475 pre-built connectors (enterprise, AI, IoT, Blockchain etc). It's used as middleware and as a rapid application platform by big companies like Circle K, Ulta Beauty, Driven Brands, Pamper

url https://www.searce.com/ urlS https://www.searce.com/
i 2482 https://www.searce.com/ ['digital transformation'] 
    NaN
url https://www.infostretch.com/ urlS https://www.infostretch.com/
i 2491 https://www.infostretch.com/ ['digital transformation'] 
 s([], )
url https://cloud.google.com/apigee urlS https://cloud.google.com/apigee
i 2496 https://cloud.google.com/apigee ['digital transformation'] 
 s([], )
url https://www.intellectsoft.net urlS https://www.intellectsoft.net
i 2511 https://www.intellectsoft.net ['digital transformation'] 
    Established in 2007, Intellectsoft evolved into an international, full-scale software solution development company with a strong focus on emerging technology like Blockchain, Augmented Reality, Big Data, Internet of Things, Artificial Intelligence, and other. For over 10 years now the company has been exploring the use of various tech innovations, helping organizations worldwide build and maintain cutting-edge IT solutions that drive real value 

url https://www.inmar.com/ urlS https://www.inmar.com/
i 2951 https://www.inmar.com/ ['digital transformation'] 
    Anyone who has redeemed a coupon, filled a prescription or returned a product, has touched Inmar. We apply technology and data science to improve outcomes for consumers and those who serve them. As a trusted intermediary for over 35 years, we have unmatched access to billions of consumer and business transactions in real time. Our analytics, platforms and services enable engagement with shoppers and patients, and optimize results. Founded in 1980, Inmar is headquartered in Winston-Salem, North Carolina with locations throughout the United States, Mexico and Canada.
url https://www.infostretch.com/ urlS https://www.infostretch.com/
i 2955 https://www.infostretch.com/ ['digital transformation'] 
 s([], )
url https://www.syniti.com/ urlS https://www.syniti.com/
i 2960 https://www.syniti.com/ ['digital transformation'] 
 s([], )
url https://www.marlabs.com urlS https://www.m

i 3938 https://securiti.ai/ ['cloud based'] 
    Securiti is an innovator of AI-Powered cybersecurity and data protection infrastructure. Our product suite is the world's first PrivacyOps platform that helps automate all major functions needed for privacy compliance in one place. It enables enterprises to give rights to people on their data, be responsible custodians of people’s data, comply with global privacy regulations and bolster their brands. The AI-Powered PrivacyOps platform is a full-stack solution that operationalizes and simplifies privacy compliance using robotic automation and a natural language interface. These include: - Sensitive Data Intelligence - DSR (Data Subject Requests) Robotic Automation - Data Mapping Automation - Consent Management - Assessment Automation - Vendor Risk Management - AUTI, Expert Bot Assistance
url https://ydrive.ai urlS https://ydrive.ai
i 3983 https://ydrive.ai ['cloud based'] 
    Ydrive is building next generation maps for use by machines an

i 4814 https://www.techforce.ai/ ['digital transformation'] 
 s([], )
url https://dxminds.com urlS https://dxminds.com
i 4825 https://dxminds.com ['digital transformation'] 
    We are DxMinds Technologies, the fastest growing digital transformation company. We have our offices set up in the silicon valley of USA, India, Peru, UAE, and Australia. We master the art of developing cutthroat digital products and solutions leveraging state of the art technologies and best industry practices. Our innovation labs set up across various nations ensures on-time delivery of precise and full-fledged products to our end customers. We showcase a wide array of engagement models like offshore development, turnkey and co-owned projects, build operate transfer models, etc. that are designed to meet customer requirements to the core. We are a team of 100+ professionals and our in-house engineers masters the art of carving out the best solutions leveraging technologies like Artificial Intelligence, Machin

i 5470 https://www.saigen.co.za/ ['cloud based'] 
 s([], )
url https://scale.com urlS https://scale.com
i 5529 https://scale.com ['document processing'] 
     Scale accelerates the development of AI applications by helping machine learning teams generate high-quality ground truth data. Our advanced LiDAR, video, and image annotation APIs allow self-driving, drone, and robotics teams at companies like Lyft, OpenAI, Zoox, Pinterest, and Airbnb focus on building differentiated models vs. labeling data.
11846    Scale accelerates the development of AI applications by helping machine learning teams generate high-quality ground truth data. Our advanced LiDAR, video, and image annotation APIs allow self-driving, drone, and robotics teams at companies like Lyft, OpenAI, Zoox, Pinterest, and Airbnb focus on building differentiated models vs. labeling data.
url https://viubox.com urlS https://viubox.com
i 5609 https://viubox.com ['digitization'] 
     ViuBox is a line of smart solutions from Sen

url https://onlinesales.ai/ urlS https://onlinesales.ai/
i 6038 https://onlinesales.ai/ ['digital transformation'] 
     OnlineSales.ai is a E-Commerce Marketing and Analytics Platform. OnlineSales.ai enables eCommerce businesses globally in their pursuit of higher sales & margins. It provides 10x better experience in acquisition & retention marketing for eCommerce.  At the core of OnlineSales AI platform, billions of data points are mined daily from across retailers’ product catalog, demand velocity, user psychographics & funnel analytics. The platform leverages this intelligence to deliver unparalleled marketing experience & automation across channels & devices to eCommerce players globally. OnlineSales.ai’s vision is to build an ‘operating system’ for eCommerce businesses globally by converging all avenues of shopper acquisition, engagement, retention, analytics & monetization on a single platform.
13114    OnlineSales.ai is a E-Commerce Marketing and Analytics Platform. OnlineSales

This is where the doc2vec routine from gensim is called. Consult gensim documentation for parameter definitions.

In [6]:
dim_vector=25
model = gensim.models.doc2vec.Doc2Vec(dm=1, vector_size=dim_vector, min_count=2, epochs=200)
model.build_vocab(train_corpus)
print('model.corpus_count',model.corpus_count)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

2022-08-11 14:22:19,595 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d25,n5,w5,mc2,s0.001,t3)', 'datetime': '2022-08-11T14:22:19.594308', 'gensim': '4.0.1', 'python': '3.6.10 |Anaconda, Inc.| (default, May  7 2020, 19:46:08) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}
2022-08-11 14:22:19,596 : INFO : collecting all words and their counts
2022-08-11 14:22:19,597 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2022-08-11 14:22:19,934 : INFO : collected 47868 word types and 6480 unique tags from a corpus of 6480 examples and 1965360 words
2022-08-11 14:22:19,934 : INFO : Creating a fresh vocabulary
2022-08-11 14:22:20,052 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=2 retains 28701 unique words (59.95863624968664%% of original 47868, drops 19167)', 'datetime': '2022-08-11T14:22:20.052285', 'gensim': '4.0.1', 'python': '3.6.10 |Anaconda, Inc.| (default, May  7 2020, 19:46:08) [MSC v

model.corpus_count 6480


2022-08-11 14:22:21,572 : INFO : EPOCH 1 - PROGRESS: at 67.53% examples, 1068672 words/s, in_qsize 5, out_qsize 0
2022-08-11 14:22:22,062 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-08-11 14:22:22,064 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-08-11 14:22:22,069 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-08-11 14:22:22,071 : INFO : EPOCH - 1 : training on 1965360 raw words (1585610 effective words) took 1.5s, 1066580 effective words/s
2022-08-11 14:22:23,062 : INFO : EPOCH 2 - PROGRESS: at 61.37% examples, 983052 words/s, in_qsize 6, out_qsize 0
2022-08-11 14:22:23,687 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-08-11 14:22:23,687 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-08-11 14:22:23,702 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-08-11 14:22:23,704 : INFO : EPOCH - 2 : training on 1965360 raw words (1585767 effec

2022-08-11 14:22:45,089 : INFO : EPOCH - 16 : training on 1965360 raw words (1586040 effective words) took 1.6s, 972554 effective words/s
2022-08-11 14:22:46,105 : INFO : EPOCH 17 - PROGRESS: at 54.24% examples, 877988 words/s, in_qsize 6, out_qsize 0
2022-08-11 14:22:46,877 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-08-11 14:22:46,877 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-08-11 14:22:46,889 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-08-11 14:22:46,890 : INFO : EPOCH - 17 : training on 1965360 raw words (1585367 effective words) took 1.8s, 882028 effective words/s
2022-08-11 14:22:47,888 : INFO : EPOCH 18 - PROGRESS: at 54.91% examples, 896754 words/s, in_qsize 6, out_qsize 0
2022-08-11 14:22:48,675 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-08-11 14:22:48,681 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-08-11 14:22:48,682 : INFO : worker t

2022-08-11 14:23:14,724 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-08-11 14:23:14,727 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-08-11 14:23:14,736 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-08-11 14:23:14,738 : INFO : EPOCH - 32 : training on 1965360 raw words (1585433 effective words) took 2.3s, 682918 effective words/s
2022-08-11 14:23:15,751 : INFO : EPOCH 33 - PROGRESS: at 40.52% examples, 664386 words/s, in_qsize 5, out_qsize 0
2022-08-11 14:23:16,753 : INFO : EPOCH 33 - PROGRESS: at 84.24% examples, 657360 words/s, in_qsize 5, out_qsize 0
2022-08-11 14:23:17,089 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-08-11 14:23:17,089 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-08-11 14:23:17,109 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-08-11 14:23:17,111 : INFO : EPOCH - 33 : training on 1965360 raw words (1585855 eff

...

2022-08-11 14:26:33,483 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-08-11 14:26:33,488 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-08-11 14:26:33,489 : INFO : EPOCH - 139 : training on 1965360 raw words (1585671 effective words) took 1.8s, 868134 effective words/s
2022-08-11 14:26:34,501 : INFO : EPOCH 140 - PROGRESS: at 53.69% examples, 874491 words/s, in_qsize 5, out_qsize 0
2022-08-11 14:26:35,303 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-08-11 14:26:35,303 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-08-11 14:26:35,319 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-08-11 14:26:35,319 : INFO : EPOCH - 140 : training on 1965360 raw words (1585795 effective words) took 1.8s, 866229 effective words/s
2022-08-11 14:26:36,331 : INFO : EPOCH 141 - PROGRESS: at 47.33% examples, 785544 words/s, in_qsize 6, out_qsize 0
2022-08-11 14:26:37,236 : INFO : work

2022-08-11 14:27:00,971 : INFO : EPOCH - 154 : training on 1965360 raw words (1585560 effective words) took 1.7s, 911003 effective words/s
2022-08-11 14:27:01,990 : INFO : EPOCH 155 - PROGRESS: at 56.31% examples, 896218 words/s, in_qsize 6, out_qsize 0
2022-08-11 14:27:02,694 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-08-11 14:27:02,703 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-08-11 14:27:02,714 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-08-11 14:27:02,715 : INFO : EPOCH - 155 : training on 1965360 raw words (1585508 effective words) took 1.7s, 911503 effective words/s
2022-08-11 14:27:03,709 : INFO : EPOCH 156 - PROGRESS: at 57.08% examples, 920056 words/s, in_qsize 6, out_qsize 0
2022-08-11 14:27:04,420 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-08-11 14:27:04,430 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-08-11 14:27:04,436 : INFO : work

2022-08-11 14:27:30,041 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-08-11 14:27:30,063 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-08-11 14:27:30,071 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-08-11 14:27:30,073 : INFO : EPOCH - 170 : training on 1965360 raw words (1586129 effective words) took 1.8s, 864555 effective words/s
2022-08-11 14:27:31,089 : INFO : EPOCH 171 - PROGRESS: at 51.03% examples, 830624 words/s, in_qsize 5, out_qsize 0
2022-08-11 14:27:31,966 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-08-11 14:27:31,969 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-08-11 14:27:31,988 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-08-11 14:27:31,990 : INFO : EPOCH - 171 : training on 1965360 raw words (1585390 effective words) took 1.9s, 829184 effective words/s
2022-08-11 14:27:33,000 : INFO : EPOCH 172 - PROGRESS: at 51.

2022-08-11 14:27:58,513 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-08-11 14:27:58,514 : INFO : EPOCH - 184 : training on 1965360 raw words (1586037 effective words) took 1.8s, 859064 effective words/s
2022-08-11 14:27:59,532 : INFO : EPOCH 185 - PROGRESS: at 54.24% examples, 876547 words/s, in_qsize 5, out_qsize 1
2022-08-11 14:28:00,342 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-08-11 14:28:00,347 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-08-11 14:28:00,353 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-08-11 14:28:00,354 : INFO : EPOCH - 185 : training on 1965360 raw words (1585337 effective words) took 1.8s, 862771 effective words/s
2022-08-11 14:28:01,361 : INFO : EPOCH 186 - PROGRESS: at 52.15% examples, 856321 words/s, in_qsize 6, out_qsize 0
2022-08-11 14:28:02,174 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-08-11 14:28:02,190 : INFO : work

2022-08-11 14:28:27,704 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-08-11 14:28:27,704 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-08-11 14:28:27,704 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-08-11 14:28:27,717 : INFO : EPOCH - 200 : training on 1965360 raw words (1584836 effective words) took 1.8s, 868250 effective words/s
2022-08-11 14:28:27,719 : INFO : Doc2Vec lifecycle event {'msg': 'training on 393072000 raw words (317101804 effective words) took 367.1s, 863711 effective words/s', 'datetime': '2022-08-11T14:28:27.719906', 'gensim': '4.0.1', 'python': '3.6.10 |Anaconda, Inc.| (default, May  7 2020, 19:46:08) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'train'}


Doc2vec has now created and trained a model for the input documents from AItotal_v2.cor.
This next section looks at the inferred vector for each document (i.e. for the
scraped text from each company's website) and finds the closest other documents.
That similarity is given by the sims matrix, sims[1][1] being the closest. An arbitrary threshold of 0.8
is used to decide whether to output to the screen and the output file similarity.txt the most similar document/url.

In [11]:
ranks = []
second_ranks = []
ftot=sum(flengths)
color1=np.zeros(ftot)

# there is simply no documentation on what xxx.words means. It is a
# gensim thing and a feature of a "TaggedDocument". By usage, a TaggedDocument
# seems to be a pair of a list of strings and an index (a single integer in a list,
# e.g. [4]). Doing .words on it gets the list of strings part.
# it is a list of topn tuples (index,closeness).


distance=np.zeros((len(train_corpus),len(train_corpus)))
inferred_vector_array=np.empty([len(train_corpus),dim_vector])

for s in range(LL):
    color1=color1+color_array[:,s]

train_corpus_np=np.array(train_corpus)

z=train_corpus
w=train_corpus_np

wlist=w.tolist()

output_file=open('similarity.txt',"w") # "w" deletes any pre-existing file
inferred_vector_array=np.empty([len(train_corpus),dim_vector])

ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)-1):
    inferred_vector=model.infer_vector(train_corpus[doc_id].words)
    inferred_vector_array[doc_id]=inferred_vector
    sims=model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid,sim in sims].index(doc_id)
    ranks.append(rank)
    second_ranks.append(sims[1])
    if sims[1][1] > 0.80:
        string=str(sims[1][1])
        line1='.'.join(train_corpus[doc_id][0][0:4]) + ' ***most like*** '\
        + '.'.join(train_corpus[sims[1][0]][0][0:4])
        output_file.write(line1)
        output_file.write("\n")
        
output_file.close()
inferred_vector_array=np.nan_to_num(inferred_vector_array,copy=False)


TSNE is t-distributed stochastic neighbor embedding. 
https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding
Basically reduces multidimensional data to 2d (or 3D) for visualization.

In [12]:
X_embedded = TSNE(n_components=2,perplexity=50).fit_transform(inferred_vector_array)
X2=np.array(X_embedded)

In [13]:
print('X2.shape',X2.shape)
print('X_embedded[0][0]',X_embedded[0],len(X_embedded))
xtest=[]
xtest.append(X_embedded[0])
print('xtest',xtest)

X2.shape (6480, 2)
X_embedded[0][0] [38.342186   4.8658814] 6480
xtest [array([38.342186 ,  4.8658814], dtype=float32)]


For plotting, if the document contains one of the keywords in search_list then it will be given
a larger marker and a color from the array color1 which is set above. The points which are small
and black in the ensuing plot have none of the keywords. Docs with more than one keyword have
the last color in the list (I think). This could use improvement.

In [14]:
MM=X2.shape[0]
X2A=[]
X2B=[]
CA=[]
CB=[]
color1L=list(color1)
for ii in range(MM):
    if color1[ii] == 0.0:
        X2A.append(X_embedded[ii])
        CA.append(color1L[ii])
    else:
        X2B.append(X_embedded[ii])
        CB.append(color1L[ii])
        
X2An=np.array(X2A)
X2Bn=np.array(X2B)
CAn=np.array(CA)
CBn=np.array(CB)
# print('X2An',X2An,'\n X2Bn',X2Bn,'\n CAn',CAn,'\n CBn',CBn)

In [15]:
plt.figure(63,figsize=(15,10))
for ii in range(CAn.shape[0]):
    CAn[ii]=38.0

plt.scatter(X2An[:,0],X2An[:,1],s=5,c='black',cmap='gist_ncar',vmin=0,vmax=100)
plt.scatter(X2Bn[:,0],X2Bn[:,1],s=40,c=CBn,cmap='gist_ncar',vmin=0,vmax=100)
plt.colorbar()
plt.xlim(-60,60)
plt.ylim(-60,60)
plt.show(63)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
counter = collections.Counter(ranks)
print(counter)